In [ ]:
#Importing all libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import xlsxwriter
from datetime import timedelta, datetime

In [ ]:
#Open trendogate website in browser

driver = webdriver.Chrome("C:/Drivers/chromedriver.exe")
driver.maximize_window()
driver.get("https://trendogate.com/")

time.sleep(2)

In [ ]:
#enter starting and ending date in month/date/year format
starting_date = 'month/date/year'
ending_date='month/date/year'
starting_date = datetime.strptime(starting_date, '%m/%d/%Y').date()
ending_date = datetime.strptime(ending_date, '%m/%d/%Y').date()



#Create an excel file
workbook = xlsxwriter.Workbook("Trendogate.xlsx")
worksheet = workbook.add_worksheet()
bold = workbook.add_format({'bold': True})

worksheet.write('A1', 'Date',bold)
worksheet.write('B1', 'Hashtag',bold)
worksheet.write('C1', 'URL',bold)
worksheet.write('D1', 'Tweets',bold)

row=1

while starting_date<=ending_date:
    
    
    #Choose date and location and search for all hashtag 

    Choose_location=Select(driver.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[1]/select"))
    Choose_location.select_by_visible_text('India')

    date=starting_date.strftime('%m/%d/%Y')
    print("date:",date)
    time.sleep(1)
    
    Choose_Date=driver.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input").send_keys(date)
    time.sleep(2)

    Search=driver.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()
    time.sleep(2)
    
    
    #Get list of all hashtag for particular day.
    div=driver.find_elements_by_xpath("//html/body/div[4]/div/div[1]/div/ul/li")
    print(len(div))
    hash_url={}
    for i in range(1, len(div) + 1):
        hashtag = driver.find_element_by_xpath("//html/body/div[4]/div/div[1]/div/ul/li[{}]/a".format(i))   
        link = hashtag.get_attribute('href')
        hash_url.update({hashtag.text:link})
    print(hash_url)
    print(len(hash_url))
    
 
    for (hashtag, url) in hash_url.items():

        #Enter date in Date column of excel file
        worksheet.write_string(row, 0, str(date))

        #Enter hashtag name in Hasgtag column of excel file
        worksheet.write_string(row, 1, str(hashtag))
        time.sleep(1)

        #Enter link of hashtag in URL column of excel file
        worksheet.write_string(row, 2, str(url))
        time.sleep(1)

        #open link of particular hashtag
        driver.get(url)
        time.sleep(2)

        #Check for more than one pages
        try:
            #for multiple pages

            next_button=driver.find_elements_by_xpath("//html/body/div[4]/div/div[1]/center[1]/nav/ul/li")
            tweets=[]
            for nxt in range(2,len(next_button)):
                page = driver.find_element_by_xpath("//html/body/div[4]/div/div[1]/center[1]/nav/ul/li[{}]/a".format(nxt))
                page_link=page.get_attribute('href')
                driver.get(page_link)
                time.sleep(2)
                paragraphs = driver.find_elements_by_tag_name('p')
                for paragraph in paragraphs:
                    tweets.append(paragraph.text)


            #Enter list of tweets in Tweets column of excel file        
            worksheet.write_string(row, 3, str(tweets))        
            row=row+1

        except:
            #for one page

            tweets=[]
            paragraphs = driver.find_elements_by_tag_name('p')
            for paragraph in paragraphs:
                tweets.append(paragraph.text)

            #Enter list of tweets in Tweets column of excel file     
            worksheet.write_string(row, 3, str(tweets))        
            row=row+1
    
    starting_date += timedelta(days=1)

workbook.close()
print("Excel file ready")